In [10]:
import pandas as pd
import numpy as np

detail = pd.read_csv('./data/detail.csv', encoding='gbk')

In [11]:
# 哑变量
data = detail.loc[:5,'dishes_name']
print(data)

data = pd.get_dummies(data)
print(data)

0     蒜蓉生蚝
1    蒙古烤羊腿
2     大蒜苋菜
3    芝麻烤紫菜
4      蒜香包
5      白斩鸡
Name: dishes_name, dtype: object
   大蒜苋菜  白斩鸡  芝麻烤紫菜  蒙古烤羊腿  蒜蓉生蚝  蒜香包
0     0    0      0      0     1    0
1     0    0      0      1     0    0
2     1    0      0      0     0    0
3     0    0      1      0     0    0
4     0    0      0      0     0    1
5     0    1      0      0     0    0


In [12]:
# 单个的连续数据，  变成离散数据， 
# print(detail['amounts'])

# 等宽法
# pd.cut 参数输入的是数字，就根据等宽法来切割，如果输入的是具体的值的范围，就根据具体的范围来分
data = pd.cut(detail['amounts'],5)
print(data.value_counts()) # 统计每个区间的数量



(0.823, 36.4]     5461
(36.4, 71.8]      3157
(71.8, 107.2]      839
(142.6, 178.0]     426
(107.2, 142.6]     154
Name: amounts, dtype: int64


In [13]:
# 等频法，每个区间的数量一致
# 1,2,3,3,3,3,5,7
# 1,2,3,3,3,3,
# 5,7

data = detail['amounts']

# 范围排名0,0.2,0.4,...,1的具体的值
w = data.quantile(np.array([0,0.2,0.4,0.6,0.8,1]))
print(type(w))
print(w)
#print(data)

# 把边界值给后面的范围
data = pd.cut(data, w)
print(data.value_counts())

<class 'pandas.core.series.Series'>
0.0      1.0
0.2     18.0
0.4     32.0
0.6     39.0
0.8     58.0
1.0    178.0
Name: amounts, dtype: float64
(18.0, 32.0]     2107
(39.0, 58.0]     2080
(32.0, 39.0]     1910
(1.0, 18.0]      1891
(58.0, 178.0]    1863
Name: amounts, dtype: int64


In [14]:
# k-means算法进行分类
from sklearn.cluster import KMeans

data = detail['amounts']

#进行聚类，分成5类
kmodel = KMeans(n_clusters = 5,random_state=123)

# 需要传入
data = data.values.reshape((len(data),1))

#训练数据
kmodel.fit(data)

#输出聚类的中心点并排序
#之所以这里使用DataFrame，是因为聚类的数据的特征值可以不止一个
c = pd.DataFrame(kmodel.cluster_centers_).sort_values(0)
#print(c)

#计算分界点
w = c.rolling(2).mean().iloc[1:]
print(type(w[0]))

#分成5份，需要6个边界值
w = [data.min()] + list(w[0]) + [data.max()]


data = detail['amounts']
data = pd.cut(data, w) # w值使用array和list都是可以的

print(data.value_counts())

<class 'pandas.core.series.Series'>
(22.31, 43.51]       3690
(43.51, 73.945]      2474
(1.0, 22.31]         2268
(73.945, 131.858]     993
(131.858, 178.0]      426
Name: amounts, dtype: int64
